In [1]:
import pandas as pd
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
captureSize = 35.8 #for tmb claculation
import seaborn as sns
sns.set(style = 'white', font_scale=1.5)


In [2]:
itrp_data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/SKCM_v2/'
tcga_data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/'
immu_data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/InnateDB/'
nano_data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/NanoString/'
ppi_data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/PPI/'

merge_data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/'

In [3]:
! ls -lh /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/*.TABLE

-rw-rw-r-- 1 was966 zitnik 3.4M Sep  1 14:32 /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/GENE.TABLE
-rw-rw-r-- 1 was966 zitnik 1.7M Sep  1 13:33 /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/PATIENTS.TABLE
-rw-rw-r-- 1 was966 zitnik 4.4G Sep  1 14:35 /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/TPM.TABLE


In [4]:
! ls -lh /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/SKCM_v2/*.TABLE

-rw-rw-r-- 1 was966 zitnik 55K Sep  1 14:56 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/SKCM_v2/PATIENTS.TABLE
-rw-rw-r-- 1 was966 zitnik 51M Sep  1 14:56 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/SKCM_v2/TPM_CORRECTION.TABLE
-rw-rw-r-- 1 was966 zitnik 51M Sep  1 14:56 /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/SKCM_v2/TPM.TABLE


In [5]:

## ITRP dataset
itrp_patient = pd.read_pickle(os.path.join(itrp_data_path, 'PATIENTS.TABLE'))
itrp_tpm = pd.read_pickle(os.path.join(itrp_data_path, 'TPM.TABLE'))
itrp_tpm_correct = pd.read_pickle(os.path.join(itrp_data_path, 'TPM_CORRECTION.TABLE'))

## TCGA dataset
tcga_gene = pd.read_pickle(os.path.join(tcga_data_path, 'GENE.TABLE'))
tcga_patient = pd.read_pickle(os.path.join(tcga_data_path, 'PATIENTS.TABLE'))
tcga_tpm = pd.read_pickle(os.path.join(tcga_data_path, 'TPM.TABLE'))

## Merge
ensid_freeman = pd.Series(dict(zip(itrp_tpm.index.map(lambda x:x.split('.')[0]), itrp_tpm.index)))
ensid_freeman.name = 'ensid_freeman'
common_genes = set(tcga_tpm.columns.map(lambda x:x.split('.')[0])) & set(itrp_tpm.index.map(lambda x:x.split('.')[0]))
common_genes = list(common_genes)
print(len(common_genes))

tcga_gene = tcga_gene.loc[common_genes]
tcga_gene = tcga_gene[tcga_gene.gene_type == 'protein_coding']
tcga_gene = tcga_gene.join(ensid_freeman)

tcga_tpm = tcga_tpm[tcga_gene.ensid_v36]
tcga_tpm.columns = tcga_gene.gene_name.tolist()

itrp_tpm = itrp_tpm.T[tcga_gene.ensid_freeman]
itrp_tpm.columns = tcga_gene.gene_name.tolist()

itrp_tpm_correct = itrp_tpm_correct.T[tcga_gene.ensid_freeman]
itrp_tpm_correct.columns = tcga_gene.gene_name.tolist()

itrp_patient = itrp_patient.loc[itrp_tpm.index]

18980


In [6]:
tcga_tpm.shape, itrp_tpm.shape, itrp_tpm_correct.shape, tcga_gene.shape

((10184, 18474), (334, 18474), (334, 18474), (18474, 5))

In [7]:
itrp_patient.shape, tcga_patient.shape

((334, 20), (10184, 22))

In [8]:
### remove genes with very low variance
def _sel_genes(x):
    var = x.var(axis=0) 
    sel_genes = var[var > 1e-8].index
    return sel_genes
sgs1 = itrp_tpm.join(itrp_patient.cohort).groupby('cohort').apply(_sel_genes)
var_gene_list = list(set.intersection(*map(set, sgs1)))
len(var_gene_list)

var = tcga_tpm.var() 
var_gene_list = var[var > 1e-4].index
len(var_gene_list)

18425

## 01. Save 18425

In [9]:
tcga_tpm_1 = np.log2(tcga_tpm[var_gene_list]+1)
itrp_tpm_1 = np.log2(itrp_tpm[var_gene_list]+1)
itrp_tpm_1_correct = np.log2(itrp_tpm_correct[var_gene_list]+1)

gene_1 = tcga_gene[tcga_gene.gene_name.isin(var_gene_list)].reset_index().set_index('gene_name')
tcga_tpm_1.shape, itrp_tpm_1.shape, tcga_patient.shape, itrp_patient.shape, gene_1.shape

((10184, 18425), (334, 18425), (10184, 22), (334, 20), (18425, 5))

In [10]:
! mkdir -p /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/18425
tcga_tpm_1.to_pickle(os.path.join(merge_data_path, '18425', 'TPM.TCGA.TABLE'))
itrp_tpm_1.to_pickle(os.path.join(merge_data_path, '18425', 'TPM.ITRP.TABLE'))
itrp_tpm_1_correct.to_pickle(os.path.join(merge_data_path, '18425', 'TPM.ITRP.TABLE.CORRECT'))

gene_1.to_pickle(os.path.join(merge_data_path, '18425', 'GENE.TABLE'))
tcga_patient.to_pickle(os.path.join(merge_data_path, '18425', 'PATIENT.TCGA.TABLE'))
itrp_patient.to_pickle(os.path.join(merge_data_path, '18425', 'PATIENT.ITRP.TABLE'))
! ls -lh /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/18425

total 1.5G
-rw-rw-r-- 1 was966 zitnik 1.4M Sep  7 17:11 GENE.TABLE
-rw-rw-r-- 1 was966 zitnik  55K Sep  7 17:11 PATIENT.ITRP.TABLE
-rw-rw-r-- 1 was966 zitnik 1.7M Sep  7 17:11 PATIENT.TCGA.TABLE
-rw-rw-r-- 1 was966 zitnik  48M Sep  7 17:11 TPM.ITRP.TABLE
-rw-rw-r-- 1 was966 zitnik  48M Sep  7 17:11 TPM.ITRP.TABLE.CORRECT
-rw-rw-r-- 1 was966 zitnik 1.4G Sep  7 17:11 TPM.TCGA.TABLE


## 02. Save 6575

In [11]:
# Immunogenes
df_immu_gene = pd.read_pickle(os.path.join(immu_data_path, 'InnateDB.TABLE'))
immu_gene_list = list(set(df_immu_gene.gene_name) & set(var_gene_list))

tcga_tpm_2 = np.log2(tcga_tpm[immu_gene_list]+1)
itrp_tpm_2 = np.log2(itrp_tpm[immu_gene_list]+1)
itrp_tpm_2_correct = np.log2(itrp_tpm_correct[immu_gene_list]+1)

gene_2 = tcga_gene[tcga_gene.gene_name.isin(immu_gene_list)].reset_index().set_index('gene_name')
mut_gene = df_immu_gene.set_index('gene_name')
mut_gene = mut_gene[mut_gene.columns[2:]]
gene_2 = gene_2.join(mut_gene)
tcga_tpm_2.shape, itrp_tpm_2.shape, tcga_patient.shape, itrp_patient.shape, gene_2.shape

((10184, 6575), (334, 6575), (10184, 22), (334, 20), (6575, 11))

In [12]:
! mkdir -p /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/6575
tcga_tpm_2.to_pickle(os.path.join(merge_data_path, '6575', 'TPM.TCGA.TABLE'))
itrp_tpm_2.to_pickle(os.path.join(merge_data_path, '6575', 'TPM.ITRP.TABLE'))
itrp_tpm_2_correct.to_pickle(os.path.join(merge_data_path, '6575', 'TPM.ITRP.TABLE.CORRECT'))
gene_2.to_pickle(os.path.join(merge_data_path, '6575', 'GENE.TABLE'))
tcga_patient.to_pickle(os.path.join(merge_data_path, '6575', 'PATIENT.TCGA.TABLE'))
itrp_patient.to_pickle(os.path.join(merge_data_path, '6575', 'PATIENT.ITRP.TABLE'))
! ls -lh /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/6575

total 579M
-rw-rw-r-- 1 was966 zitnik 537K Sep  7 17:11 GENE.TABLE
-rw-rw-r-- 1 was966 zitnik  55K Sep  7 17:11 PATIENT.ITRP.TABLE
-rw-rw-r-- 1 was966 zitnik 1.7M Sep  7 17:11 PATIENT.TCGA.TABLE
-rw-rw-r-- 1 was966 zitnik  17M Sep  7 17:11 TPM.ITRP.TABLE
-rw-rw-r-- 1 was966 zitnik  17M Sep  7 17:11 TPM.ITRP.TABLE.CORRECT
-rw-rw-r-- 1 was966 zitnik 512M Sep  7 17:11 TPM.TCGA.TABLE


### 03. Save 735

In [13]:
# NanoString genes
df_nano_gene = pd.read_pickle(os.path.join(nano_data_path, 'NanoString.TABLE'))
nano_gene_list = list(set(df_nano_gene.index) & set(var_gene_list))
len(df_immu_gene), len(immu_gene_list), len(df_nano_gene), len(nano_gene_list)

(7394, 6575, 756, 735)

In [14]:
tcga_tpm_3 = np.log2(tcga_tpm[nano_gene_list]+1)
itrp_tpm_3 = np.log2(itrp_tpm[nano_gene_list]+1)
itrp_tpm_3_correct = np.log2(itrp_tpm_correct[nano_gene_list]+1)

gene_3 = tcga_gene[tcga_gene.gene_name.isin(nano_gene_list)].reset_index().set_index('gene_name')
gene_3 = gene_3.join(df_nano_gene)
tcga_tpm_3.shape, itrp_tpm_3.shape, tcga_patient.shape, itrp_patient.shape, gene_3.shape

((10184, 735), (334, 735), (10184, 22), (334, 20), (735, 10))

In [16]:
! mkdir -p /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/735
tcga_tpm_3.to_pickle(os.path.join(merge_data_path, '735', 'TPM.TCGA.TABLE'))
itrp_tpm_3.to_pickle(os.path.join(merge_data_path, '735', 'TPM.ITRP.TABLE'))
itrp_tpm_3_correct.to_pickle(os.path.join(merge_data_path, '735', 'TPM.ITRP.TABLE.CORRECT'))


gene_3.to_pickle(os.path.join(merge_data_path, '735', 'GENE.TABLE'))
tcga_patient.to_pickle(os.path.join(merge_data_path, '735', 'PATIENT.TCGA.TABLE'))
itrp_patient.to_pickle(os.path.join(merge_data_path, '735', 'PATIENT.ITRP.TABLE'))
! ls -lh /n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/735

total 68M
-rw-rw-r-- 1 was966 zitnik 101K Sep  7 17:13 GENE.TABLE
-rw-rw-r-- 1 was966 zitnik  55K Sep  7 17:13 PATIENT.ITRP.TABLE
-rw-rw-r-- 1 was966 zitnik 1.7M Sep  7 17:13 PATIENT.TCGA.TABLE
-rw-rw-r-- 1 was966 zitnik 1.9M Sep  7 17:13 TPM.ITRP.TABLE
-rw-rw-r-- 1 was966 zitnik 1.9M Sep  7 17:13 TPM.ITRP.TABLE.CORRECT
-rw-rw-r-- 1 was966 zitnik 1.9M Sep  7 17:11 TPM.ITRP.TABLE.CRORRECT
-rw-rw-r-- 1 was966 zitnik  58M Sep  7 17:13 TPM.TCGA.TABLE
